# Brute force - Code pour plusieurs jours (MAJ suite à sécurité Nitter)

In [71]:
#pip install selenium
#pip install webdriver_manager
#pip install bs4

In [114]:
import datetime
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen, Request
import urllib.parse

In [115]:
# Créer une liste pour stocker les résultats de chaque jour
results = []

In [116]:
# Définir la date de début et la date de fin
start_date = datetime.date(2023, 1, 1)
end_date = datetime.date(2023, 1, 2)

In [117]:
# Chargement du dictionnaire des mots
dict_mot = pd.read_csv('PPD_Dictionnaire_des_mots.csv',sep=';')
print(dict_mot.to_string())

                          Mot  Fréquence
0       transition écologique        339
1                  production        128
2                     métiers        101
3                 compétences         96
4                    stockage         89
5                      enjeux         75
6                      impact         75
7          matières premières         74
8                 entreprises         71
9                 énergétique         59
10                    énergie         55
11                    chaleur         55
12                       plan         51
13                  industrie         45
14      développement durable         43
15                   capacité         42
16                  véhicules         42
17                     master         41
18                  batteries         40
19                     energy         37
20               consommation         36
21                        for         34
22                        all         34
23              

In [118]:
# Filtre manuel pour récupérer que les mots sur lesquels on peut scrapper
mots_a_conserver = [
    'transition écologique',
    'transition ecologique',
    'transition énergétique',
    'transition energetique',
    'énergies nouvelles',
    'nouvelles énergies',
    'énergies renouvelables'
    'matières premières',
    'matieres premieres',
    'enjeu environnemental',
    'enjeux environnementaux',
    'développement durable',
    'efioul',
    'e-fioul',
    'carburant de synthese',
    'carburant de synthèse',
    'biocarburant',
    'bio-carburant',
    'rechauffement climatique',
    'rechauffement climatique',
    'voiture électrique',
    'voitures électriques',
    'véhicule électrique',
    'véhicules électriques',
    'pompe à chaleur',
    'isolation du logement',
    'biochar',
    'éolienne en mer',
    'éoliennes en mer',
    'éolienne offshore',
    'éoliennes offshore',
    'hydrogène',
    'consommation énergétique',
    'sobriété énergétique',
    'entreprises vertes'
]

# Filtrer le dictionnaire de mots en ne conservant que les mots spécifiques
dict_mot_final = dict_mot[dict_mot['Mot'].isin(mots_a_conserver)]
dict_mot_final = dict_mot_final.reset_index(drop=True)
dict_mot_final

,Mot,Fréquence
0,transition écologique,339
1,développement durable,43
2,entreprises vertes,12
3,transition ecologique,999
4,transition énergétique,999
5,transition energetique,999
6,énergies nouvelles,999
7,nouvelles énergies,999
8,matieres premieres,999
9,enjeu environnemental,999


In [119]:
# Construire la liste des mots à rechercher
mots_a_rechercher = dict_mot_final['Mot'].tolist()

# Remplacement des ' 'par '+' pour prératation à la requête 
mots_a_rechercher = [mots.replace(' ', '+') for mots in mots_a_rechercher]
mots_a_rechercher

['transition+écologique',
 'développement+durable',
 'entreprises+vertes',
 'transition+ecologique',
 'transition+énergétique',
 'transition+energetique',
 'énergies+nouvelles',
 'nouvelles+énergies',
 'matieres+premieres',
 'enjeu+environnemental',
 'enjeux+environnementaux',
 'efioul',
 'e-fioul',
 'carburant+de+synthese',
 'carburant+de+synthèse',
 'biocarburant',
 'bio-carburant',
 'rechauffement+climatique',
 'voiture+électrique',
 'voitures+électriques',
 'véhicule+électrique',
 'véhicules+électriques',
 'pompe+à+chaleur',
 'isolation+du+logement',
 'biochar',
 'éolienne+en+mer',
 'éoliennes+en+mer',
 'éolienne+offshore',
 'éoliennes+offshore',
 'hydrogène',
 'consommation+énergétique',
 'sobriété+énergétique']

In [120]:
# Boucle sur chaque mot à rechercher
for mot in mots_a_rechercher:
    # Réinitialiser la date de début pour chaque nouveau mot
    start_date_mot = start_date
    
    # Boucle sur chaque jour
    while start_date_mot <= end_date:
        since_date_mot = start_date_mot.strftime("%Y-%m-%d")
        until_date_mot = (start_date_mot + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
        
        # Construire l'URL de la requête avec le mot et les dates du jour en cours
        query = urllib.parse.quote(mot)
        url = f"https://nitter.net/search?f=tweets&q={query}&since={since_date_mot}&until={until_date_mot}&near="
        print(url)

        # Requête HTTP & récupération du contenu de la page (+ ajout d'un header pour contourner la sécurité Nitter)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        req = Request(url, headers=headers)
        response = urlopen(req)
        soup = BeautifulSoup(response.read(), 'html.parser')

        # extraire les informations de chaque tweet pour le jour en cours
        tweet_contents = []
        tweet_dates = []
        tweet_links = []
        tweet_names = []
        tweet_names_id = []
        comment_stats = []
        retweet_stats = []
        quote_stats = []
        heart_stats = []

        for tweet in soup.find_all('div', {'class': 'timeline-item'}):
            # contenu du tweet
            content = tweet.find('div', {'class': 'tweet-content'}).text.strip()
            tweet_contents.append(content)

            # date du tweet
            date = tweet.find('span', {'class': 'tweet-date'}).a['title']
            tweet_dates.append(date)

            # lien du tweet
            link = tweet.find('a', {'class': 'tweet-link'})['href']
            tweet_links.append(link)

            # nom (title) du tweet
            name = tweet.find('a', {'class': 'fullname'})['title']
            tweet_names.append(name)

            # nom (id) du tweet
            name_id = tweet.find('a', {'class': 'username'})['title']
            tweet_names_id.append(name_id)

            # statistiques
            stats = tweet.find_all('div', {'class': 'icon-container'})
            comment_stats.append(stats[0].text.strip())
            retweet_stats.append(stats[1].text.strip())
            quote_stats.append(stats[2].text.strip())
            heart_stats.append(stats[3].text.strip())

        # ajouter les informations du jour en cours à la liste de résultats
        result = pd.DataFrame({
            'Content': tweet_contents,
            'Date': tweet_dates,
            'Link': tweet_links,
            'Utilisateur': tweet_names,
            'Utilisateur_id': tweet_names_id,
            'Comments': comment_stats,
            'Retweets': retweet_stats,
            'Quotes': quote_stats,
            'Hearts': heart_stats
        })
        results.append(result)

        # Passer à la prochaine date
        start_date_mot += datetime.timedelta(days=1)

https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-01-01&until=2023-01-02&near=
https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-01-02&until=2023-01-03&near=
https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-01-01&until=2023-01-02&near=
https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-01-02&until=2023-01-03&near=
https://nitter.net/search?f=tweets&q=entreprises%2Bvertes&since=2023-01-01&until=2023-01-02&near=
https://nitter.net/search?f=tweets&q=entreprises%2Bvertes&since=2023-01-02&until=2023-01-03&near=
https://nitter.net/search?f=tweets&q=transition%2Becologique&since=2023-01-01&until=2023-01-02&near=
https://nitter.net/search?f=tweets&q=transition%2Becologique&since=2023-01-02&until=2023-01-03&near=
https://nitter.net/search?f=tweets&q=transition%2B%C3%A9nerg%C3%A9tique&since=2023-01-01&until=2023-01-02&near=
https://nitter.net/search?f=tweets&q=transition%2B%C3%A9nerg%C3%A9

In [121]:
# Concaténer les résultats de chaque jour dans un unique DataFrame
df = pd.concat(results)

In [122]:
df

,Content,Date,Link,Utilisateur,Utilisateur_id,Comments,Retweets,Quotes,Hearts
0,"Ah, en effet...\nLa transition écologique est ...","Jan 1, 2023 · 11:52 PM UTC",/PaysagesIroise/status/1609699207169941505#m,Sauvegarde Paysages d'Iroise,@PaysagesIroise,,,,1
1,Méthode pour une transition écologique dans le...,"Jan 1, 2023 · 11:29 PM UTC",/InfosEducation/status/1609693388776906752#m,Education & Formation,@InfosEducation,,,,
2,Enseignement agricole et transition écologique...,"Jan 1, 2023 · 11:03 PM UTC",/VIENSflashmober/status/1609686785084493826#m,Etat d'urgence,@VIENSflashmober,,,,
3,la pollueuse de notre planète qui a des intérê...,"Jan 1, 2023 · 10:26 PM UTC",/LIBERTE1311/status/1609677427223007232#m,eddy,@LIBERTE1311,,,,1
4,Quand la transition écologique bouscule la pol...,"Jan 1, 2023 · 10:21 PM UTC",/PACOLLOT/status/1609676117555781634#m,Pierre-Alain Collot,@PACOLLOT,,,,
...,...,...,...,...,...,...,...,...,...
15,Couplé à une RTX 4090 Ti et probablement à un ...,"Jan 2, 2023 · 8:21 PM UTC",/ffunivers/status/1610008375810748416#m,Orange pressé 💉💉 BioNtech 💉 NIH/Moderna 💉#BA5,@ffunivers,,,,1
16,"Dans ma boîte, pour la sobriété énergétique il...","Jan 2, 2023 · 8:16 PM UTC",/FREDDABSURDE/status/1610007050033020928#m,⛳ FREDD ABSURDE🍌,@FREDDABSURDE,,1,,3
17,“Le pilotage à distance permet vraiment d’agir...,"Jan 2, 2023 · 8:00 PM UTC",/B_SMART_TV/status/1610002995839737858#m,B SMART,@B_SMART_TV,,,,
18,Pour cause : \n_augmentation fréquente de la s...,"Jan 2, 2023 · 7:54 PM UTC",/LuigiBetach/status/1610001599509708801#m,Luigi b 黄油,@LuigiBetach,,,,1


In [123]:
# Fonction pour nettoyer les dates
def clean_date(date_string):
    if pd.isnull(date_string):
        return date_string
    cleaned_string = date_string.replace("¬∑", "")
    cleaned_string = date_string.replace("Â·", "")
    cleaned_string = date_string.replace("Ā", "")
    cleaned_string = date_string.replace("·", "")
    return cleaned_string.strip()

# Nettoyer la colonne 'Date'
df['Date'] = df['Date'].apply(lambda x: clean_date(x))

# Convertir la colonne 'Date' en format datetime
df['Date'] = pd.to_datetime(df['Date'], format="%b %d, %Y %I:%M %p UTC", errors='coerce')

# Créer les colonnes 'Jour', 'Mois' et 'Année'
df['Jour'] = df['Date'].dt.day
df['Mois'] = df['Date'].dt.month
df['Année'] = df['Date'].dt.year

# Convertir les colonnes 'Jour', 'Mois' et 'Année' en entiers
df['Jour'] = df['Jour'].apply(lambda x: str(int(x)) if not pd.isnull(x) else '')
df['Mois'] = df['Mois'].apply(lambda x: str(int(x)) if not pd.isnull(x) else '')
df['Année'] = df['Année'].apply(lambda x: str(int(x)) if not pd.isnull(x) else '')

# Convertir la colonne 'Date' en format JJ/MM/AAAA
df['Date'] = df['Date'].dt.strftime("%d/%m/%Y")


In [124]:
df = df.reindex(columns=['Content', 'Date', 'Jour', 'Mois', 'Année', 'Link', 'Utilisateur', 'Utilisateur_id', 'Comments', 'Retweets', 'Quotes', 'Hearts'])

In [125]:
df

,Content,Date,Jour,Mois,Année,Link,Utilisateur,Utilisateur_id,Comments,Retweets,Quotes,Hearts
0,"Ah, en effet...\nLa transition écologique est ...",01/01/2023,1,1,2023,/PaysagesIroise/status/1609699207169941505#m,Sauvegarde Paysages d'Iroise,@PaysagesIroise,,,,1
1,Méthode pour une transition écologique dans le...,01/01/2023,1,1,2023,/InfosEducation/status/1609693388776906752#m,Education & Formation,@InfosEducation,,,,
2,Enseignement agricole et transition écologique...,01/01/2023,1,1,2023,/VIENSflashmober/status/1609686785084493826#m,Etat d'urgence,@VIENSflashmober,,,,
3,la pollueuse de notre planète qui a des intérê...,01/01/2023,1,1,2023,/LIBERTE1311/status/1609677427223007232#m,eddy,@LIBERTE1311,,,,1
4,Quand la transition écologique bouscule la pol...,01/01/2023,1,1,2023,/PACOLLOT/status/1609676117555781634#m,Pierre-Alain Collot,@PACOLLOT,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
15,Couplé à une RTX 4090 Ti et probablement à un ...,02/01/2023,2,1,2023,/ffunivers/status/1610008375810748416#m,Orange pressé 💉💉 BioNtech 💉 NIH/Moderna 💉#BA5,@ffunivers,,,,1
16,"Dans ma boîte, pour la sobriété énergétique il...",02/01/2023,2,1,2023,/FREDDABSURDE/status/1610007050033020928#m,⛳ FREDD ABSURDE🍌,@FREDDABSURDE,,1,,3
17,“Le pilotage à distance permet vraiment d’agir...,02/01/2023,2,1,2023,/B_SMART_TV/status/1610002995839737858#m,B SMART,@B_SMART_TV,,,,
18,Pour cause : \n_augmentation fréquente de la s...,02/01/2023,2,1,2023,/LuigiBetach/status/1610001599509708801#m,Luigi b 黄油,@LuigiBetach,,,,1


In [17]:
import csv

df.to_csv(
    'PPD_ALL_Scrapping.csv',
    sep=';',
    index=False,
    encoding='utf-8',
    #quoting=csv.QUOTE_NONE,
    #escapechar='\\'
)

# Selenium - Code avec 1 jour (demo)

In [41]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [42]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://nitter.net/search?f=tweets&q=transition+%C3%A9nerg%C3%A9tique+energetique&since=2023-04-20&until=2023-04-21&near=")
html = driver.page_source

In [43]:
soup = BeautifulSoup(html, 'html.parser')

In [44]:
# extraire les informations de chaque tweet
tweet_contents = []
tweet_dates = []
tweet_links = []
tweet_names = []
comment_stats = []
retweet_stats = []
quote_stats = []
heart_stats = []

for tweet in soup.find_all('div', {'class': 'timeline-item'}):
    # contenu du tweet
    content = tweet.find('div', {'class': 'tweet-content'}).text.strip()
    tweet_contents.append(content)
    
    # date du tweet
    date = tweet.find('span', {'class': 'tweet-date'}).a['title']
    tweet_dates.append(date)
    
    # lien du tweet
    link = tweet.find('a', {'class': 'tweet-link'})['href']
    tweet_links.append(link)
    
    # nom (title) du tweet
    name = tweet.find('a', {'class': 'fullname'})['title']
    tweet_names.append(name)
    
    # statistiques
    stats = tweet.find_all('div', {'class': 'icon-container'})
    comment_stats.append(stats[0].text.strip())
    retweet_stats.append(stats[1].text.strip())
    quote_stats.append(stats[2].text.strip())
    heart_stats.append(stats[3].text.strip())

In [45]:
# créer un dataframe pandas à partir des listes d'informations
df = pd.DataFrame({
    'Content': tweet_contents,
    'Date': tweet_dates,
    'Link': tweet_links,
    'Utilisateur': tweet_names,
    'Comments': comment_stats,
    'Retweets': retweet_stats,
    'Quotes': quote_stats,
    'Hearts': heart_stats
})

In [46]:
df

,Content,Date,Link,Utilisateur,Comments,Retweets,Quotes,Hearts
0,"RATP (Bus)\n\n🌿Depuis le 19/04/23, la ligne 14...","Apr 20, 2023 · 11:14 PM UTC",/chanstransports/status/1649189726593732608#m,Claude-Henry & ses périples,2,6,1,47
1,"Le rapport rendu ce vendredi à @AgnesRunacher,...","Apr 20, 2023 · 10:45 PM UTC",/ChamboncelBen/status/1649182448842833921#m,Benoît Chamboncel,,,22,1
2,📢 Défis de l'employabilité avec la transition ...,"Apr 20, 2023 · 10:19 PM UTC",/IFDD_OIF/status/1649176003158192128#m,IFDD,,2,,5
3,« C’est un pied de nez au Québec. On se fait d...,"Apr 20, 2023 · 9:22 PM UTC",/seblemire/status/1649161573867352066#m,Sébastien Lemire,2,,,1
4,Chère @_EvaSas En #Allemagne ils n'ont pas att...,"Apr 20, 2023 · 8:56 PM UTC",/PIERREMAURIN/status/1649154996355624967#m,Pierre Maurin,,1,,2
5,📃 Le 1er conseil d’administration de la Fondat...,"Apr 20, 2023 · 7:11 PM UTC",/FondationOpenC/status/1649128606319214593#m,Fondation OPEN-C,1,1,,4
6,Métaux stratégiques de l'Afrique pour la trans...,"Apr 20, 2023 · 6:56 PM UTC",/energiemines/status/1649124953923387417#m,Magazine Energie,,,,
7,il est juste que les acteurs publics[…]soient ...,"Apr 20, 2023 · 6:49 PM UTC",/JCondamine/status/1649123244178276383#m,Jean Condamine 🇫🇷 🇪🇸 🇬🇧 👍🍻🍺🤔🧐😉👌🥂🍾,2,5,,22
8,Roland Lescure et de la ministre de la transit...,"Apr 20, 2023 · 6:20 PM UTC",/DanielM11626471/status/1649115758109007875#m,Daniel Moore 🇷🇺🇷🇺🇷🇺,1,1,,4
9,"Plus de 10 millions de morts, au moins 500 000...","Apr 20, 2023 · 6:20 PM UTC",/EpopoYanick/status/1649115722583359492#m,Epopo yanick,,1,,1


In [47]:
# Fonction pour convertir la date dans le bon format
def convert_date(date_string):
    date_obj = datetime.datetime.strptime(date_string.split(' · ')[0], "%b %d, %Y")
    return date_obj.strftime("%d/%m/%Y")

# Convertir la colonne 'Date' en format datetime
df['Date'] = pd.to_datetime(df['Date'].apply(lambda x: x.split(' · ')[0]), format='%b %d, %Y')

# Créer les colonnes 'Jour', 'Mois' et 'Année'
df['Jour'] = df['Date'].dt.day
df['Mois'] = df['Date'].dt.month
df['Année'] = df['Date'].dt.year

# Convertir la colonne 'Date' en format JJ/MM/AAAA
df['Date'] = df['Date'].apply(lambda x: convert_date(x.strftime("%b %d, %Y")))

In [48]:
df = df.reindex(columns=['Content', 'Date', 'Jour', 'Mois', 'Année', 'Link', 'Utilisateur','Comments', 'Retweets', 'Quotes', 'Hearts'])

In [49]:
df

,Content,Date,Jour,Mois,Année,Link,Utilisateur,Comments,Retweets,Quotes,Hearts
0,"RATP (Bus)\n\n🌿Depuis le 19/04/23, la ligne 14...",20/04/2023,20,4,2023,/chanstransports/status/1649189726593732608#m,Claude-Henry & ses périples,2,6,1,47
1,"Le rapport rendu ce vendredi à @AgnesRunacher,...",20/04/2023,20,4,2023,/ChamboncelBen/status/1649182448842833921#m,Benoît Chamboncel,,,22,1
2,📢 Défis de l'employabilité avec la transition ...,20/04/2023,20,4,2023,/IFDD_OIF/status/1649176003158192128#m,IFDD,,2,,5
3,« C’est un pied de nez au Québec. On se fait d...,20/04/2023,20,4,2023,/seblemire/status/1649161573867352066#m,Sébastien Lemire,2,,,1
4,Chère @_EvaSas En #Allemagne ils n'ont pas att...,20/04/2023,20,4,2023,/PIERREMAURIN/status/1649154996355624967#m,Pierre Maurin,,1,,2
5,📃 Le 1er conseil d’administration de la Fondat...,20/04/2023,20,4,2023,/FondationOpenC/status/1649128606319214593#m,Fondation OPEN-C,1,1,,4
6,Métaux stratégiques de l'Afrique pour la trans...,20/04/2023,20,4,2023,/energiemines/status/1649124953923387417#m,Magazine Energie,,,,
7,il est juste que les acteurs publics[…]soient ...,20/04/2023,20,4,2023,/JCondamine/status/1649123244178276383#m,Jean Condamine 🇫🇷 🇪🇸 🇬🇧 👍🍻🍺🤔🧐😉👌🥂🍾,2,5,,22
8,Roland Lescure et de la ministre de la transit...,20/04/2023,20,4,2023,/DanielM11626471/status/1649115758109007875#m,Daniel Moore 🇷🇺🇷🇺🇷🇺,1,1,,4
9,"Plus de 10 millions de morts, au moins 500 000...",20/04/2023,20,4,2023,/EpopoYanick/status/1649115722583359492#m,Epopo yanick,,1,,1


# Selenium - Code pour plusieurs jours

In [5]:
pip install webdriver_manager

In [53]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [52]:
# Date de début de la recherche
start_date = datetime.date(2023, 5, 1)

# Date de fin de la recherche
end_date = datetime.date(2023, 5, 2)

# Créer la liste des dates à parcourir
delta = datetime.timedelta(days=1)
dates = []
while start_date <= end_date:
    dates.append(start_date)
    start_date += delta

In [54]:
# Initialiser le webdriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Liste des données à récupérer
tweet_contents = []
tweet_dates = []
tweet_links = []
tweet_names = []
comment_stats = []
retweet_stats = []
quote_stats = []
heart_stats = []

In [55]:
# Boucle pour parcourir les pages pour chaque date
for date in dates:
    url = f"https://nitter.net/search?f=tweets&q=transition+%C3%A9nerg%C3%A9tique+energetique&since={date}&until={date+delta}&near="
    driver.get(url)
    while True:
        # Extraire les informations de chaque tweet sur la page
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        for tweet in soup.find_all('div', {'class': 'timeline-item'}):
            # Contenu du tweet
            content = tweet.find('div', {'class': 'tweet-content'}).text.strip()
            tweet_contents.append(content)
            
            # Date du tweet
            date = tweet.find('span', {'class': 'tweet-date'}).a['title']
            tweet_dates.append(date)
            
            # Lien du tweet
            link = tweet.find('a', {'class': 'tweet-link'})['href']
            tweet_links.append(link)
            
            # Nom (title) du tweet
            name = tweet.find('a', {'class': 'fullname'})['title']
            tweet_names.append(name)
            
            # Statistiques
            stats = tweet.find_all('div', {'class': 'icon-container'})
            comment_stats.append(stats[0].text.strip())
            retweet_stats.append(stats[1].text.strip())
            quote_stats.append(stats[2].text.strip())
            heart_stats.append(stats[3].text.strip())
        
        # Cliquer sur le bouton "Load more" si présent
        try:
            #load_more_button = driver.find_element_by_class_name("show-more").find_element_by_tag_name("a")
            #load_more_button = driver.find_elements(By.CLASS_NAME, "show-more")
            #load_more_button=load_more_button.find_elements(By.TAG_NAME, "a")
            #print(load_more_button)
            #load_more_button.click()
            #print(load_more_button[0].text)
            #driver.execute_script("arguments[0].click();", load_more_button);
            
            # get element 
            element = driver.find_element_by_link_text("Load more")
            print(element)
            # click the element
            element.click()
            
        except:
            break

In [56]:
# Créer un dataframe pandas avec les données récupérées
df = pd.DataFrame({
    'Content': tweet_contents,
    'Date': tweet_dates,
    'Link': tweet_links,
    'Utilisateur': tweet_names,
    'Comments': comment_stats,
    'Retweets': retweet_stats,
    'Quotes': quote_stats,
    'Hearts': heart_stats
})

In [57]:
# Fonction pour convertir la date dans le bon format
def convert_date(date_string):
    date_obj = datetime.datetime.strptime(date_string.split(' · ')[0], "%b %d, %Y")
    return date_obj.strftime("%d/%m/%Y")

# Convertir la colonne 'Date' en format datetime
df['Date'] = pd.to_datetime(df['Date'].apply(lambda x: x.split(' · ')[0]), format='%b %d, %Y')

# Créer les colonnes 'Jour', 'Mois' et 'Année'
df['Jour'] = df['Date'].dt.day
df['Mois'] = df['Date'].dt.month
df['Année'] = df['Date'].dt.year

# Convertir la colonne 'Date' en format JJ/MM/AAAA
df['Date'] = df['Date'].apply(lambda x: convert_date(x.strftime("%b %d, %Y")))

In [58]:
df = df.reindex(columns=['Content', 'Date', 'Jour', 'Mois', 'Année', 'Link', 'Utilisateur','Comments', 'Retweets', 'Quotes', 'Hearts'])

In [59]:
df

,Content,Date,Jour,Mois,Année,Link,Utilisateur,Comments,Retweets,Quotes,Hearts
0,Pourquoi le Maroc est important pour la transi...,01/05/2023,1,5,2023,/Rozlbrahim/status/1653155503642402821#m,인생은 계속됩니다,1,,,1
1,dans le sable du désert - et des plans ambitie...,01/05/2023,1,5,2023,/Rozlbrahim/status/1653155505852801024#m,인생은 계속됩니다,1,,,1
2,Repenser la transition énergétique : Le menson...,01/05/2023,1,5,2023,/PEGE_french/status/1653127278828691457#m,Mösl Roland,,,,
3,"Là j'approuve, ça n'est pas une oeuvre d'art q...",01/05/2023,1,5,2023,/marco_il_biondo/status/1653119911894327302#m,Marco il Biondo,,,,
4,"Oui, mais non puisque l'Allemagne se donne les...",01/05/2023,1,5,2023,/Djoanarcho/status/1653111091021463552#m,BZH 🏴‍☠️🏴,,,,
5,Pas la peine de demander qui vous suivez polit...,01/05/2023,1,5,2023,/ceditpro/status/1653101214333034514#m,cedric,,,,
6,Participez à la transition énergétique avec ce...,01/05/2023,1,5,2023,/GreenBois_fr/status/1653093514144538632#m,Green Bois,,,,
7,"mes félicitations ! votre livre, très utile po...",01/05/2023,1,5,2023,/michel_Halpes/status/1653090639108620289#m,Michel Cranga climat et responsabilité,2,,,3
8,"Besoin en investissement, en travailleurs, con...",01/05/2023,1,5,2023,/FR_Conversation/status/1653090552382992384#m,The Conversation France,,,,2
9,"""transition écologique"", ou encore ""transition...",01/05/2023,1,5,2023,/5gLinky/status/1653084728046632978#m,Stop Linky-5G Montpellier,,,,2


# Brute force acteurs - Code pour plusieurs jours (MAJ suite à sécurité Nitter)

In [ ]:
df = 